In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
import pickle as pkl
from tensorflow import keras
import tensorflow.keras.backend as K
from keras.models import Sequential,Model
from keras.models import Sequential 
from keras.layers import Dense, Dropout, Flatten, Input, Reshape, BatchNormalization, Bidirectional, LSTM, Multiply, Activation, Normalization, TimeDistributed
from keras.layers import Conv3D, Conv2D, Conv1D, MaxPool3D, MaxPool2D, MaxPool1D, AvgPool3D, AvgPool2D, AvgPool1D, GlobalMaxPool3D, Attention
from keras.layers import GlobalMaxPool2D, GlobalMaxPool1D, SpatialDropout3D, SpatialDropout2D, SpatialDropout1D, GlobalAvgPool3D, MultiHeadAttention
from keras.layers import GlobalAvgPool2D, GlobalAvgPool1D, SeparableConv2D, SeparableConv1D, Add, Concatenate, LeakyReLU, ELU, Activation, PReLU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
from keras.utils import np_utils
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score, precision_recall_fscore_support
from sklearn.utils.class_weight import compute_class_weight
import gc

# **Data Loading**

In [2]:
from google.colab import drive
drive.mount('/content/drive/')
gc.collect()

Mounted at /content/drive/


11

In [3]:
input_path='/content/drive/MyDrive/data_preprocessed_python/'

In [4]:
def baseline_remove(XR):
  XT = XR[:, 384:]
  XB_ = (XR[:, :1*128] + XR[:, 1*128:2*128] + XR[:, 2*128:3*128])/3.0
  for i in range(60):
    XT[:, i*128:(i+1)*128] = XT[:, i*128:(i+1)*128] - XB_
  return XT


In [5]:
eeg_signal = []
valence = []
arousal = []
window_size = 384
skip = 384

for person in range(7,8):
  print('Person No.' + str(person))
  
  
  address = input_path+'s'+str(person)+'.dat'

  with open(address, 'rb') as file:
    data = pkl.load(file, encoding = 'latin1')

  eeg = data['data']
  label = data['labels']
  
  # Assigning classes
  label[label<5] = 0
  label[label>=5] = 1

  val = label.T[0] # Valence label
  aro = label.T[1] # Arousal label

  del data, label
  # Iterating through 40 vidoes/trials
  for i in range(40):
    sig = eeg[i][:32, :]
    sig = baseline_remove(sig)
    # Segmenting into 3 seconds (384 timesteps) windows without overlap
    start = 0
    while start + window_size <=sig.shape[1]:
      eeg_signal.append(sig[:, start:start+window_size])
      valence.append(val[i])
      arousal.append(aro[i])
      start += skip 
  del eeg, val, aro, sig
eeg_signal = np.reshape(eeg_signal,[-1,32,384,1])
data = np.asarray(eeg_signal, dtype = np.float32) # Using 32 bit floating point value to save memory
del eeg_signal
valence = np.asarray(valence, dtype = np.int8)
arousal = np.asarray(arousal, dtype = np.int8)

print(data.shape)
print(valence.shape, valence[valence == 0].shape, valence[valence == 1].shape)
print(arousal.shape, arousal[arousal == 0].shape, arousal[arousal == 1].shape)

valence = np_utils.to_categorical(valence)
arousal = np_utils.to_categorical(arousal)
gc.collect()

Person No.7
(800, 32, 384, 1)
(800,) (240,) (560,)
(800,) (300,) (500,)


0

In [6]:
 X_train, x_test, Y_train, y_test = train_test_split(data, valence, test_size=0.1, random_state=4)
 print(X_train.shape,Y_train.shape)

(720, 32, 384, 1) (720, 2)


In [13]:
def dilation_1DCNN_block(x, dilation_rate):
    y = Conv1D(filters=8, kernel_size=7, strides=1, padding='causal', dilation_rate=dilation_rate)(x)
    y = PReLU(alpha_initializer=tf.initializers.constant(0.2))(y)
    y = Normalization()(y)
    y = Conv1D(filters=8, kernel_size=7, strides=1, padding='causal', dilation_rate=dilation_rate)(y)
    y = PReLU(alpha_initializer=tf.initializers.constant(0.2))(y)
    y = Normalization()(y)
    return y

def causal_dilation_1DCNN_block(x):
    x = Conv1D(filters=8, kernel_size=11, strides=1, padding='causal', dilation_rate=1)(x)
    y1 = dilation_1DCNN_block(x, 1)
    y2 = dilation_1DCNN_block(Add()([x, y1]), 2)
    y3 = dilation_1DCNN_block(Add()([y1, y2]), 4)
    y4 = dilation_1DCNN_block(Add()([y2, y3]), 8)
    x = Add()([y1, y2, y3, y4])
    x = Dropout(0.2)(x)
    #x = Reshape((-1, x.shape[1]*x.shape[2]), name='Reshape_1')(x)
    x = LSTM(32, activation='tanh', recurrent_activation='sigmoid', use_bias=True, kernel_initializer='glorot_uniform',
           recurrent_initializer='orthogonal', bias_initializer='zeros', unit_forget_bias=True, kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4), 
           bias_regularizer=regularizers.l2(1e-4), activity_regularizer=regularizers.l2(1e-5), kernel_constraint=None, 
           dropout=0.2, recurrent_dropout=0.2, return_sequences=True, return_state=True, go_backwards=True, name='LSTM_1')(x)
    x = LSTM(32, activation='elu', recurrent_activation='sigmoid', use_bias=True, kernel_initializer='glorot_uniform',
           recurrent_initializer='orthogonal', bias_initializer='zeros', unit_forget_bias=True, kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4), 
           bias_regularizer=regularizers.l2(1e-4), activity_regularizer=regularizers.l2(1e-5), kernel_constraint=None, 
           dropout=0.1, recurrent_dropout=0.1, go_backwards=True)(x)
    y = Dropout(0.2)(x)
    #y = Activation('softmax', name = 'Activation_2')(y)
    #x = Multiply(name='Multiply_2')([x, y])
    x = Flatten(name='Flatten_2')(x)
    x = Dense(2, activation='softmax', name='Dense_3') (x)
   
    return x

In [14]:
def model_form():
  inp = Input((384,1))
  out = causal_dilation_1DCNN_block(inp)
  model = Model(inputs=inp,outputs=out)
  return model
 
model = model_form()
model.summary()
opt=Adam(learning_rate=0.0001)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=["accuracy"])

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 384, 1)]     0           []                               
                                                                                                  
 conv1d_21 (Conv1D)             (None, 384, 8)       96          ['input_4[0][0]']                
                                                                                                  
 conv1d_22 (Conv1D)             (None, 384, 8)       456         ['conv1d_21[0][0]']              
                                                                                                  
 p_re_lu_18 (PReLU)             (None, 384, 8)       3072        ['conv1d_22[0][0]']              
                                                                                            

In [15]:
batch_size = 10
epochs = 200
data_train = []
data_test = []
## Training data shaping
for i in range(X_train.shape[1]):
  data1 = X_train[:,i,:,:]
  data_test_1 = x_test[:,i,:,:]
  data_train.append(data1)
  data_test.append(data_test_1)
data_train = np.array(data_train)
data_test = np.array(data_test)
print(data_train.shape,data_test.shape)


(32, 720, 384, 1) (32, 80, 384, 1)


In [ ]:
weight_save_path='/content/drive/My Drive/Results/Weight/'
for i in range(4):
  data_div = data_train[i,:,:,:]
  x_train, x_val, y_train, y_val = train_test_split(data_div, Y_train, test_size=0.1, random_state=4)
  model.fit(x_train, y_train, epochs=1, batch_size=batch_size, verbose=1, validation_data=(x_val, y_val),shuffle=True)
  model.save_weights(weight_save_path+'channel '+ str(i+1)+ '.h5')
      

35/65 [===============>..............] - ETA: 1:25 - loss: 0.6393 - accuracy: 0.6914

In [ ]:
## Data Concatenation
data_train_merge = []
data_test_merge = []

for i in range(4):
  data_load = data_train[i,:,:,:]
  test_data_load = data_test[i,:,:,:] 
  model.load_weights(weight_save_path+'channel '+str(i+1)+'.h5')
  final_model = Model(inputs=model.input,outputs=model.layers[-5].output)
  data_train_predict = final_model.predict(data_load)
  data_test_predict = final_model.predict(test_data_load)
  data_train_merge.append(data_train_predict)
  data_test_merge.append(data_test_predict)

data_train_merge = np.array(data_train_merge)
data_test_merge = np.array(data_test_merge)

## Reshaping is required if it is to be converted in 2D format

## Model training after data concatenation (with Attention and Dense Layer)
#model_last = 


# **Previous Model**

In [ ]:
def channel_wise_attention(x):
  y = AvgPool2D(pool_size=(1,384), name='AvgPool2D_1')(x)
  y = Flatten(name='Flatten_1')(y)
  y = Dense(5,activation='tanh', name='Dense_1') (y)
  y = Dense(32,activation='softmax', name='Dense_2') (y)
  x = Multiply(name='Multiply_1')([x, tf.expand_dims(tf.expand_dims(y, axis=2), axis=2)])
  return x

def CNN_LSTM_Self_Attention(x):
  x = Conv2D(filters = 40, kernel_size = (32, 45), strides = (1, 1), padding = 'valid', name = 'Conv2D_1')(x)
  x = BatchNormalization(name='BatchNormalization_1')(x)
  x = Activation('elu', name = 'Activation_1')(x)
  x = MaxPool2D(pool_size=(1, 75), strides = 10, name='MaxPool2D_1')(x)
  x = Reshape((-1, x.shape[1]*x.shape[2]*x.shape[3]), name='Reshape_1')(x)
  x = LSTM(64, activation='tanh', recurrent_activation='sigmoid', use_bias=True, kernel_initializer='glorot_uniform',
           recurrent_initializer='orthogonal', bias_initializer='zeros', unit_forget_bias=True, kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4), 
           bias_regularizer=regularizers.l2(1e-4), activity_regularizer=regularizers.l2(1e-5), kernel_constraint=None, 
           dropout=0.2, recurrent_dropout=0.2, return_sequences=True, return_state=True, go_backwards=True, name='LSTM_1')(x)
  x = LSTM(64, activation='tanh', recurrent_activation='sigmoid', use_bias=True, kernel_initializer='glorot_uniform',
           recurrent_initializer='orthogonal', bias_initializer='zeros', unit_forget_bias=True, kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4), 
           bias_regularizer=regularizers.l2(1e-4), activity_regularizer=regularizers.l2(1e-5), kernel_constraint=None, 
           dropout=0.2, recurrent_dropout=0.2, return_sequences=True, go_backwards=True, name='LSTM_2')(x)
  y = MultiHeadAttention(num_heads=8, key_dim=8, name = 'MultiHeadAttention_1')(x, x)
  y = Activation('softmax', name = 'Activation_2')(y)
  x = Multiply(name='Multiply_2')([x, y])
  x = Flatten(name='Flatten_2')(x)
  x = Dense(2, activation='softmax', name='Dense_3') (x)
  return x

  def get_model() :
  input_shape = (data.shape[1], data.shape[2],1)
  a = Input(input_shape, name='Input')
  out = channel_wise_attention(a)
  out = CNN_LSTM_Self_Attention(out)
  model = Model(a, out)
  opt = keras.optimizers.Adam(learning_rate=1e-4)
  model.compile(loss=keras.losses.categorical_crossentropy, metrics=["accuracy"], optimizer=opt)
  return model
model = get_model()
print(model.summary())


In [ ]:
batch_size = 10
epochs = 200
kfold = KFold(n_splits=10, random_state=1, shuffle=True)
gc.collect()

7249

In [ ]:
def call_class_weights(yt):
  y_integers = np.argmax(yt, axis=1)
  class_weights = compute_class_weight(class_weight = 'balanced', classes = np.unique(y_integers), y = y_integers)
  d_class_weights = dict(enumerate(class_weights))
  return d_class_weights

# **Valence**

In [ ]:
#valence
X_train, x_test, Y_train, y_test = train_test_split(data, valence, test_size=0.1, random_state=4)
print(X_train.shape,x_test.shape,Y_train.shape,y_test.shape)

(720, 32, 384, 1) (80, 32, 384, 1) (720, 2) (80, 2)


In [ ]:
val_res = {'accuracy': [], 'f1_score': [], 'confusion_matrix': []}
foldNum=0
#model = get_model()
for train_index, val_index in kfold.split(X_train, Y_train):
  foldNum = foldNum + 1
  model = get_model()
  print("Results for fold",foldNum)
  x_train, x_val = X_train[train_index], X_train[val_index]
  y_train, y_val = Y_train[train_index], Y_train[val_index]
  class_weight = call_class_weights(y_train)
  model.fit(x_train, y_train, epochs=epochs, class_weight=class_weight, batch_size=batch_size, verbose=1, validation_data=(x_val, y_val),shuffle=True)
  acc = model.evaluate(x_test, y_test)
  print(acc)
  val_res['accuracy'].append(acc)
  pred = model.predict(x_test)
  f1scr = f1_score(y_test.argmax(1), pred.argmax(1), average='macro')
  print(f1scr)
  val_res['f1_score'].append(f1scr)
  val_res['confusion_matrix'].append(confusion_matrix(y_test.argmax(1), pred.argmax(1)))
  gc.collect() # Garbage collecter
  del x_train, x_val, y_train, y_val, acc, f1scr
  gc.collect()

Results for fold 1
Epoch 1/200
65/65 [==============================] - 18s 44ms/step - loss: 0.8519 - accuracy: 0.5386 - val_loss: 0.8410 - val_accuracy: 0.7083
Epoch 2/200
65/65 [==============================] - 2s 33ms/step - loss: 0.8319 - accuracy: 0.6821 - val_loss: 0.8218 - val_accuracy: 0.7083
Epoch 3/200
65/65 [==============================] - 2s 32ms/step - loss: 0.8134 - accuracy: 0.7855 - val_loss: 0.8038 - val_accuracy: 0.7639
Epoch 4/200
65/65 [==============================] - 2s 31ms/step - loss: 0.7964 - accuracy: 0.7608 - val_loss: 0.7860 - val_accuracy: 0.8194
Epoch 5/200
65/65 [==============================] - 2s 32ms/step - loss: 0.7743 - accuracy: 0.8657 - val_loss: 0.7427 - val_accuracy: 0.8056
Epoch 6/200
65/65 [==============================] - 2s 32ms/step - loss: 0.7046 - accuracy: 0.9074 - val_loss: 0.6214 - val_accuracy: 0.8333
Epoch 7/200
65/65 [==============================] - 2s 32ms/step - loss: 0.6224 - accuracy: 0.9460 - val_loss: 0.5667 - val_acc

In [ ]:
acrc = np.array(val_res['accuracy']).mean(axis=0)
f1scr = np.array(val_res['f1_score']).mean(axis=0)
print("Accuracy  : {}".format(acrc[1]))
print("F1_Score  : {}".format(f1scr))
cmatrix = np.array(val_res['confusion_matrix']).mean(axis=0)
c_matrix = cmatrix/np.sum(cmatrix, axis=1).reshape(2,1)
import seaborn as sns
figure = plt.figure(figsize=(9, 9))
sns.heatmap(c_matrix, annot=True,cmap=plt.cm.Blues)
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

# **Arousal**

In [ ]:
#arousal
X_train, x_test, Y_train, y_test = train_test_split(data,arousal, test_size=0.1, random_state=4)
print(X_train.shape,x_test.shape,Y_train.shape,y_test.shape)

(7456, 200, 256, 1) (1864, 200, 256, 1) (7456, 2) (1864, 2)


In [ ]:
val_res = {'accuracy': [], 'f1_score': [], 'confusion_matrix': []}
foldNum=0
#model = get_model()
for train_index, val_index in kfold.split(X_train, Y_train):
  foldNum = foldNum + 1
  model = get_model()
  print("Results for fold",foldNum)
  x_train, x_val = X_train[train_index], X_train[val_index]
  y_train, y_val = Y_train[train_index], Y_train[val_index]
  class_weight = call_class_weights(y_train)
  model.fit(x_train, y_train, epochs=epochs, class_weight=class_weight, batch_size=batch_size, verbose=1, validation_data=(x_val, y_val),shuffle=True)
  acc = model.evaluate(x_test, y_test)
  print(acc)
  val_res['accuracy'].append(acc)
  pred = model.predict(x_test)
  f1scr = f1_score(y_test.argmax(1), pred.argmax(1), average='macro')
  print(f1scr)
  val_res['f1_score'].append(f1scr)
  val_res['confusion_matrix'].append(confusion_matrix(y_test.argmax(1), pred.argmax(1)))
  gc.collect() # Garbage collecter
  del x_train, x_val, y_train, y_val, acc, f1scr
  gc.collect()

In [ ]:
acrc = np.array(val_res['accuracy']).mean(axis=0)
f1scr = np.array(val_res['f1_score']).mean(axis=0)
print("Accuracy  : {}".format(acrc[1]))
print("F1_Score  : {}".format(f1scr))
cmatrix = np.array(val_res['confusion_matrix']).mean(axis=0)
c_matrix = cmatrix/np.sum(cmatrix, axis=1).reshape(2,1)
import seaborn as sns
figure = plt.figure(figsize=(9, 9))
sns.heatmap(c_matrix, annot=True,cmap=plt.cm.Blues)
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

In [ ]:
def FCNN_block(x):
  x = np.reshape(-1,384,1)
  x = Conv1D(32,kernel_size=11,strides=2,padding='causal',activation='relu')(x)
  x = Conv1D(32,kernel_size=7,strides=2,padding='causal',activation='PReLu')(x)
  x = Normalization()(x)
  x = Conv1D(32,kernel_size=7,padding='causal',activation='PRelu')(x)
  x = Normalization()(x)

  x1 = Conv1D(64,kernel_size=1,strides=2,activation='relu')
  x1 = AvgPool1D(pool_size=2,name='AvgPool_1_CNN')(x1)
  x2 = Conv1D(64,kernel_size=3,strides=2,activation='relu')
  x2 = AvgPool1D(pool_size=2,name='AvgPool_2_CNN')(x2)
  x2 = Conv1D(64,kernel_size=3,name='AvgPool_3_CNN')(x2)
  x3 = Conv1D(64,kernel_size=5,strides=2,activation='relu')
  x3 = Conv1D(64,kernel_size=5,activation='relu')(x3)
  xc = Concatenate()[x1,x2,x3]
  xp = Conv1D(64,kernel_size=1,activation='PRelu')(xc)
  xp = Normalization()(xp)
  

  return x
